In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# from fabulous import text as ft

In [3]:
import pprint as pprint
import seaborn as sns
# from fabulous.color import *

In [4]:
df_Original=pd.read_csv("train.csv")

In [5]:
df_Original

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,794,1,0.5,1,0,1,2,0.8,106,6,...,1222,1890,668,13,4,19,1,1,0,0
1996,1965,1,2.6,1,0,0,39,0.2,187,4,...,915,1965,2032,11,10,16,1,1,1,2
1997,1911,0,0.9,1,1,1,36,0.7,108,8,...,868,1632,3057,9,1,5,1,1,0,3
1998,1512,0,0.9,0,4,1,46,0.1,145,5,...,336,670,869,18,10,19,1,1,1,0


In [6]:
Independent_Attributes=df_Original[df_Original.corr()["price_range"].nlargest(10).index[1:]]
Dependent_Attribute=df_Original.iloc[:,-1]

In [7]:
from sklearn.preprocessing import StandardScaler
def normalize(Independent_Attributes):
        print("Mean and Standard Deviation Before")
        print(Independent_Attributes.mean(axis=0), Independent_Attributes.std(axis=0))
        
        sc=StandardScaler()
        XScaled = sc.fit_transform(Independent_Attributes)
        
        print("Mean and Standard Deviation After")
        print(XScaled.mean(axis=0).round(4), XScaled.std(axis=0))
        return XScaled

In [ ]:
from sklearn.neural_network import MLPClassifier
N_ob=MLPClassifier(hidden_layer_size=(10,20,30))
def Neural_Network(Independent_Attributes,Dependent_Attribute):
  X=Independent_Attributes.values
  Y=Dependent_Attribute.values
  print("*************Normalization/Standardization*************")
  XScaled = normalize(X) 

  skf=StratifiedKFold(n_splits=5, shuffle=True, random_state=2)
  acc=[]

  y_ori = np.array([], dtype=int)

  y_pre= np.array([], dtype=int)
  net_mat=np.zeros((4, 4))
  for train_index, test_index in skf.split(X,Y):
      
      X_train=X[train_index]
      X_test=X[test_index]
      Y_train=Y[train_index]
      Y_test=Y[test_index]  
      print(bold(green("Shape of X_train:-   {}   ||   X_test:-   {}   || Y_train:-  {}  ||  Y_test:-   {}  ".format(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)))) 
      print(bold(magenta("Train:-   {}   ||  Test:-    {} ".format(np.bincount(Y[train_index]),np.bincount(Y[test_index])))))
      
      Kn = KNeighborsClassifier(n_neighbors=10)
      Kn.fit(X_train, Y_train)
      Y_testPred = Kn.predict(X_test)

      y_ori=np.hstack((y_ori,Y_test))
      
  
      y_pre=np.hstack((y_pre,Y_testPred))
       
      testAccuracy = metrics.accuracy_score(Y_test, Y_testPred)
      
      
      print(bold(red("Test Accuracy", testAccuracy*100)))
      acc.append(testAccuracy)

      roc_score=roc_auc_score(Y_test, Kn.predict_proba(X_test), multi_class='ovr')

      print(bold(yellow("ROC_AUC_SCORE: ",roc_score)))
    

      matrix1= confusion_matrix(Y_test, Y_testPred)
      # sum of the total confusion matirx
      net_mat=net_mat+matrix1
     
      
      plot_confusion_matrix(matrix1,show_normed=True, colorbar=True, show_absolute=True)   
      
      plt.show()
  avg_accuracy=(sum(acc) / len(acc))*100
  print(bold(yellow_bg("average accuracy:  ", avg_accuracy)))       

  net_mat = net_mat.astype('int')
  print(bold(cyan(net_mat)))
  plot_confusion_matrix(net_mat,show_normed=True, colorbar=True, show_absolute=True, cmap='Blues')   

  plt.show()
   
  roc_score=roc_auc_score(Y,Kn.predict_proba(X), multi_class='ovr')
  print(bold(yellow_bg("ROC_AUC_SCORE: ",roc_score)))
 
 
  
  print(bold("Classification Report:\n"))
  report=classification_report(y_ori, y_pre,output_dict=True)
  report_Df=pd.DataFrame(report)
  print(bold(blue(report_Df)))
  sns.heatmap(report_Df.T,annot=True)
  return avg_accuracy
avg_accuracy_NN=Neural_Network(Independent_Attributes,Dependent_Attribute)